In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import math


In [2]:
df = pd.read_csv('/Users/ericjr/Documents/GitHub/Group10_CohortB_AlgorithmFinalProject/3D_VisualVar_10M.pbbs', delim_whitespace=True, header=None, skiprows=1)
df.columns = ['x', 'y', 'z'] 

In [3]:
df_sampled = df.head(200)  # Sampling the data
points = list(df_sampled.itertuples(index=False, name=None))



In [4]:
#df_clean = df.apply(pd.to_numeric, errors='coerce').dropna()


In [5]:
# converting the cleaned df to a list of tuples
#points = list(df_clean.itertuples(index=False, name=None))


In [6]:
df_sampled.head(10)


,x,y,z
0,44263.0,10849.0,49131.0
1,44313.0,10806.0,49069.0
2,44214.0,10772.0,49133.0
3,44235.0,10824.0,49072.0
4,44256.0,10941.0,49145.0
5,44345.0,10850.0,49142.0
6,44224.0,10895.0,49058.0
7,44188.0,10810.0,49168.0
8,44304.0,10790.0,49140.0
9,44177.0,10873.0,49107.0


In [7]:
df.head()

,x,y,z
0,44263.0,10849.0,49131.0
1,44313.0,10806.0,49069.0
2,44214.0,10772.0,49133.0
3,44235.0,10824.0,49072.0
4,44256.0,10941.0,49145.0


In [8]:
df_sampled.tail()


,x,y,z
195,44423.0,10767.0,49171.0
196,44334.0,10815.0,49140.0
197,44299.0,10775.0,49173.0
198,44432.0,10745.0,49198.0
199,44319.0,10735.0,49298.0


In [9]:
df_sampled.head()


,x,y,z
0,44263.0,10849.0,49131.0
1,44313.0,10806.0,49069.0
2,44214.0,10772.0,49133.0
3,44235.0,10824.0,49072.0
4,44256.0,10941.0,49145.0


In [10]:
df_sampled.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       200 non-null    float64
 1   y       200 non-null    float64
 2   z       200 non-null    float64
dtypes: float64(3)
memory usage: 4.8 KB


In [11]:

# Calculate Euclidean distance between two 3D points
def dist(p1, p2):
    # Use the Euclidean distance formula for 3D space
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 + (p1[2] - p2[2]) ** 2)

# Brute force approach to find closest point pairs
def brute_force(points, min_dist):
    closest_pairs = set()  # Initializing an empty set to store closest point pairs
    n = len(points)  # Number of points
    for i in range(n):
        for j in range(i + 1, n):  # Compare each point with every other point
            d = dist(points[i], points[j])  # Calculate distance between points
            if d < min_dist:  # If the distance is less than the current minimum
                min_dist = d  # Update minimum distance
                closest_pairs = {frozenset({points[i], points[j]})}  # Reset closest pairs( to remove duplicates)
            elif d == min_dist:  # If distance equals the current minimum
                closest_pairs.add(frozenset({points[i], points[j]}))  # Add pair to closest pairs
    return closest_pairs, min_dist

# Find closest pairs in a strip of points
def strip_closest(strip, min_dist):
    closest_pairs = set()
    strip.sort(key=lambda point: point[1])  # Sort strip by y-coordinate
    for i in range(len(strip)):
        for j in range(i + 1, min(i + 7, len(strip))):
            d = dist(strip[i], strip[j])
            if d < min_dist:
                min_dist = d
                closest_pairs = {frozenset({strip[i], strip[j]})}
            elif d == min_dist:
                closest_pairs.add(frozenset({strip[i], strip[j]}))
    return closest_pairs, min_dist

# Recursive function to find closest pair of points
def closest_pair_rec(points_x, points_z, points_y, min_dist):
    if len(points_x) <= 3:  # Base case to if there 3 or fewer points, then use brute force
        return brute_force(points_x, float('inf'))
    
    # Dividing the points around the median point
    mid = len(points_x) // 2
    mid_point = points_x[mid]

    # Spliting the  points into left and right halves based on x-coordinate
    pl_x, pr_x = points_x[:mid], points_x[mid:]
    pl_y, pr_y = [p for p in points_y if p[0] <= mid_point[0]], [p for p in points_y if p[0] > mid_point[0]]
    pl_z, pr_z = [p for p in points_z if p[0] <= mid_point[0]], [p for p in points_z if p[0] > mid_point[0]]

    # Recursively find closest pairs in the left and right halves
    closest_pairs_left, dist_left = closest_pair_rec(pl_x, pl_z, pl_y, float('inf'))
    closest_pairs_right, dist_right = closest_pair_rec(pr_x, pr_z, pr_y, float('inf'))

    # finding the smallest distance
    min_dist = min(dist_left, dist_right)
    closest_pairs = closest_pairs_left.union(closest_pairs_right)

    # Create a strip around the median point and find closest pairs within the strip
    strip = [p for p in points_y if abs(p[0] - mid_point[0]) < min_dist]
    closest_pairs_strip, strip_dist = strip_closest(strip, min_dist)
    if strip_dist < min_dist:
        closest_pairs = closest_pairs_strip
    else:
        closest_pairs = closest_pairs.union(closest_pairs_strip)

    return closest_pairs, min_dist

# Main function to find closest pairs of points in 3D
def closest_pair_3d(points, collision_threshold):
    if not points:  # If there are no points 
        return set()  # Return an empty set

    # Sort points by each coordinate
    points_x = sorted(points, key=lambda p: p[0])
    points_y = sorted(points, key=lambda p: p[1])
    points_z = sorted(points, key=lambda p: p[2])

    # Find closest pairs using the recursive function
    closest_pairs, _ = closest_pair_rec(points_x, points_z, points_y, collision_threshold)
    return closest_pairs

In [12]:
collision_threshold = 10000
closest_collision_pairs = closest_pair_3d(points, collision_threshold)


In [13]:
closest_collision_pairs


{frozenset({(44301.0, 10774.0, 49141.0), (44305.0, 10774.0, 49147.0)})}

In [14]:
#def verify_points_within_threshold(points, threshold):
 #   for i in range(len(points)):
  #      for j in range(i + 1, len(points)):
   #         if dist(points[i], points[j]) < threshold:
    #            print(f"Points within threshold found: {points[i]} and {points[j]}")
     #           return True
   # print("No points within the specified threshold.")
    #return False


#verify_points_within_threshold(points, collision_threshold)


In [15]:
x_values, y_values, z_values = zip(*points)  # Unpacking point coordinates for plotting



In [16]:
#3D scatter plot for all points
fig = go.Figure(data=[go.Scatter3d(x=x_values, y=y_values, z=z_values,
                                   mode='markers',
                                   marker=dict(size=5, opacity=0.8, color='blue'),
                                   name='All Points')])


In [17]:
if closest_collision_pairs:
    for pair in closest_collision_pairs:
        (x1, y1, z1), (x2, y2, z2) = pair
        px, py, pz = [x1, x2], [y1, y2], [z1, z2]  # coordinates for plotting

        fig.add_trace(go.Scatter3d(x=px, y=py, z=pz,
                                   mode='markers+lines',
                                   marker=dict(color='red', size=5),
                                   name='Closest Pair/Collision'))

In [18]:
#fig.update_layout(margin=dict(l=0, r=0, b=0, t=0),
 #                 scene=dict(
  #                    xaxis_title='X-coordinate',
   #                   yaxis_title='Y-coordinate',
    #                  zaxis_title='Z-coordinate'),
     #             title='3D Collision Detection Visualization')



In [19]:
fig.update_layout(title='3D Points with Closest Pairs Highlighted',
                  scene=dict(xaxis_title='X-axis',
                             yaxis_title='Y-axis',
                             zaxis_title='Z-axis'),
                  margin=dict(l=0, r=0, b=0, t=50))